In [ ]:
# This cell is added by sphinx-gallery

%matplotlib inline

import mrinversion
print(f'You are using mrinversion v{mrinversion.__version__}')


# 0.05 Li2O • 0.95 SiO2 MAS-ETA


The following example is an application of the statistical learning method in
determining the distribution of the Si-29 echo train decay constants in glasses.

Import all relevant packages.



In [ ]:
import csdmpy as cp
import matplotlib.pyplot as plt
import numpy as np

from mrinversion.kernel import relaxation
from mrinversion.linear_model import LassoFistaCV, TSVDCompression

plt.rcParams['pdf.fonttype'] = 42   # For using plots in Illustrator

def plot2D(csdm_object, **kwargs):
    plt.figure(figsize=(4, 3))
    csdm_object.plot(**kwargs)
    plt.tight_layout()
    plt.show()

## Dataset setup

### Import the dataset

Load the dataset as a CSDM data-object.



In [ ]:
# The 2D MAF dataset in csdm format
domain = "https://www.ssnmr.org/sites/default/files/mrsimulator"
filename = f"{domain}/MAS_SE_PIETA_5%25Li2O_FT.csdf"
data_object = cp.load(filename)

# Inversion only requires the real part of the complex dataset.
data_object = data_object.real
sigma = 1110.521  # data standard deviation

# Convert the MAS dimension from Hz to ppm.
data_object.dimensions[0].to("ppm", "nmr_frequency_ratio")
plot2D(data_object,cmap="gist_ncar_r")

There are two dimensions in this dataset. The dimension at index 0, the horizontal
dimension in the figure, is the MAS dimension, while the dimension at
index 1, the vertical dimension, is the signal decay from relaxation.

### Prepping the data for inversion
**Step-1: Data Alignment**

When using the csdm objects with the ``mrinversion`` package, the dimension at index
0 must be the dimension undergoing the linear inversion. In this example, we
invert the signal decay from relaxation, that is, dimension-1. The first step is to
swap the axes using a data transpose.



In [ ]:
data_object = data_object.T

**Step-2: Optimization**

Notice, that the signal from the 2D T2-MAS dataset occupies a small fraction of the
two-dimensional grid. Though you may choose to proceed with the inversion
directly onto this dataset, it is not computationally optimum. For optimum
performance, trim the dataset to the region of relevant signals. Use the appropriate
array indexing/slicing to select the signal region.



In [ ]:
data_object_truncated = data_object[:, 1220:-1220]
plot2D(data_object_truncated,cmap="gist_ncar_r")

## Linear Inversion setup

### Dimension setup

In a generic linear-inverse problem, one needs to define two sets of dimensions---
the dimensions undergoing a linear transformation, and the dimensions onto which
the inversion method transforms the data. For T2 inversion, the two sets of
dimensions are the signal decay time dimension (``kernel dimension``) and the
reciprocal T2 (``inverse dimension``).



In [ ]:
data_object_truncated.dimensions[0].to("s")  # set coordinates to 's'
kernel_dimension = data_object_truncated.dimensions[0]

### Generating the kernel

Use the :class:`~mrinversion.kernel.relaxation.T2` class to generate a T2 object
and then use its ``kernel`` method to generate the T2 relaxation kernel..



In [ ]:
relaxT2 = relaxation.T2(
    kernel_dimension=kernel_dimension,
    inverse_dimension=dict(
        count=32, minimum="1e-3 s", maximum="1e4 s", scale="log", label="log (T2 / s)"
    ),
)
inverse_dimension = relaxT2.inverse_dimension
K = relaxT2.kernel(supersampling=20)

### Data Compression

Data compression is optional but recommended. It may reduce the size of the
inverse problem and, thus, further computation time.



In [ ]:
new_system = TSVDCompression(K, data_object_truncated)
compressed_K = new_system.compressed_K
compressed_s = new_system.compressed_s

print(f"truncation_index = {new_system.truncation_index}")

## Solving the inverse problem

### FISTA LASSO cross-validation

We solve the inverse Laplace problem using the statistical learning ``FISTALassoCV``
method over a range of λ values and determine the best T2 parameter distribution for
the given 2D T2-MAS dataset.



In [ ]:
# setup the pre-defined range of alpha and lambda values
lambdas = 10 ** (-4 + 5 * (np.arange(32) / 31))

# setup the smooth lasso cross-validation class
s_lasso = LassoFistaCV(
    lambdas=lambdas,  # A numpy array of lambda values.
    sigma=sigma,  # data standard deviation
    folds=5,  # The number of folds in n-folds cross-validation.
    inverse_dimension=inverse_dimension,  # previously defined inverse dimensions.
)

# run the fit method on the compressed kernel and compressed data.
s_lasso.fit(K=compressed_K, s=compressed_s)

### The optimum hyper-parameters

Use the :attr:`~mrinversion.linear_model.LassoFistaCV.hyperparameters` attribute of
the instance for the optimum hyper-parameters, $\lambda$, determined from the
cross-validation.



In [ ]:
print(s_lasso.hyperparameters)

### The cross-validation curve



In [ ]:
plt.figure(figsize=(4, 3))
s_lasso.cv_plot()
plt.tight_layout()
plt.show()

### The optimum solution



In [ ]:
f_sol = s_lasso.f

levels = np.arange(15) / 15 + 0.1
plt.figure(figsize=(3.85, 2.75))  # set the figure size
ax = plt.subplot(projection="csdm")
cb=ax.contourf(f_sol / f_sol.max(), levels=levels, cmap="jet_r")
ax.set_ylim(-70, -130)
ax.set_xlim(-3, 2.5)
plt.title("5Li:95Si")
ax.set_xlabel("$\log(\lambda^{-1}\,/\,$s)")
ax.set_ylabel("Frequency / ppm")
plt.grid(linestyle="--", alpha=0.75)
plt.colorbar(cb,ticks=[0,.1,.2,.3,.4,.5,.6,.7,.8,.9,1.0,1.1])
plt.tight_layout()
plt.savefig("5Li-95Si.pdf")
plt.show()

### The fit residuals

To calculate the residuals between the data and predicted data(fit), use the
:meth:`~mrinversion.linear_model.LassoFistaCV.residuals` method, as follows,



In [ ]:
residuals = s_lasso.residuals(K=K, s=data_object_truncated)
plot2D(residuals,cmap="gist_ncar_r")

The standard deviation of the residuals is



In [ ]:
residuals.std()

### Saving the solution

To serialize the solution (nuclear shielding tensor parameter distribution) to a
file, use the `save()` method of the CSDM object, for example,



In [ ]:
f_sol.save("5Li-95Si-T2_inverse.csdf")  # save the solution
residuals.save("5Li-95Si-T2-residue.csdf")  # save the residuals